In [1]:
import matplotlib.pyplot as plt
plt.style.use(['seaborn-notebook', 'seaborn-white', 'seaborn-ticks'])
from mpl_toolkits.mplot3d import Axes3D

from random import random

import numpy as np

import mlgrad.model as model
import mlgrad.func as func
import mlgrad.regular as regular
import mlgrad.loss as loss
import mlgrad.weights as weights
import mlgrad.averager as averager
import mlgrad.avragg as avragg

from mlgrad import averager_it, averager_fg, fg, erm_fg, sg, erm_sg, irgd, erm_irgd, erisk, mrisk

/tmp/ipykernel_101636/4244370690.py:2: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use(['seaborn-notebook', 'seaborn-white', 'seaborn-ticks'])
/tmp/ipykernel_101636/4244370690.py:2: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use(['seaborn-notebook', 'seaborn-white', 'seaborn-ticks'])
/tmp/ipykernel_101636/4244370690.py:2: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain a

ModuleNotFoundError: No module named 'mlgrad.regular'

In [ ]:
loss_func_sq = loss.ErrorLoss(func.Square())
loss_func_abs = loss.ErrorLoss(func.Absolute())
loss_func_sqrt = loss.ErrorLoss(func.Sqrt(1.0))
loss_func_logsq = loss.ErrorLoss(func.LogSquare(1.0))

In [ ]:
def sigma_neuron_layer2(n_input, n_output):
    outfunc = func.Arctang()
    layer = model.SigmaNeuronModelLayer(outfunc, n_input, n_output)
    return layer

In [ ]:
def create_nn(n_input, n_hidden):
    layer1 = sigma_neuron_layer2(n_input, n_hidden[0])
    layer2 = sigma_neuron_layer2(n_hidden[0], n_hidden[1])
    layer3 = sigma_neuron_layer2(n_hidden[1], n_hidden[2])

    body = model.FFNetworkModel()
    body.add(layer1)
    body.add(layer2)
    body.add(layer3)

    head = model.LinearModel(n_hidden[2])
    ffnn = model.FFNetworkFuncModel(head, body)
    ffnn.allocate()
    return ffnn

In [ ]:
def generate_nn(n_input, n_hidden):
    nn_orig = create_nn(n_input, n_hidden)
    for layer in nn_orig.body.layers:
        layer_param = layer.param
        n = len(layer_param)
        random_param = 8*np.random.random(n)-4
        for i in range(n):
            layer_param[i] = random_param[i]
    head_param = nn_orig.head.param
    head_param[0] = 0.
    for i in range(n_hidden[-1]):
        head_param[i+1] = (-1.)**i /((i+1.))
    return nn_orig


In [ ]:
result = {}

n_input = 2
n_hidden = (7, 7, 3)

In [ ]:
nn_orig = generate_nn(n_input, n_hidden)
print(nn_orig.param.base)

In [ ]:
Q = 40
q = Q / 100.

N = 50
M = int(q*N*N)

In [ ]:
xrange = np.linspace(-10., 10., N)
yrange = np.linspace(-10., 10., N)
xxrange, yyrange= np.meshgrid(xrange, yrange, indexing='xy')
#iirange, jjrange= np.meshgrid(xrange, yrange, indexing='ij')
X = np.c_[xxrange.ravel(), yyrange.ravel()]
Y = np.array(nn_orig.evaluate_all(X))
Y_s = Y + 0.01*(2*np.random.random(N*N)-1)
print(np.max(Y))
YY = Y_s.reshape(xxrange.shape)
I = np.random.permutation(np.arange(0,N*N))[:M]
Y_o = Y_s.copy()
for i in I:
    yi = Y_o[i]
    s = random()
    Y_o[i] += s * abs(yi) * 3
YY_o = Y_o.reshape(xxrange.shape)

# plt.figure(figsize=(12,4))
# plt.subplot(1,2,1)
# cb1 = plt.imshow(YY, extent=(-7.,7.,-7.,7.), origin='bottom', interpolation='none')
# plt.colorbar(cb1)
# plt.subplot(1,2,2)
# cb2 = plt.imshow(YY_o, extent=(-7.,7.,-7.,7.), origin='bottom', interpolation='none')
# plt.colorbar(cb2)
# plt.show()

plt.figure(figsize=(12,5))
fig = plt.gcf()
ax1 = fig.add_subplot(1, 2, 1, projection='3d')
surf1 = ax1.plot_surface(xxrange, yyrange, YY, cmap=plt.cm.Greys)
plt.title("Outliers: 0%")
ax2 = fig.add_subplot(1, 2, 2, projection='3d')
surf2 = ax2.plot_surface(xxrange, yyrange, YY_o, cmap=plt.cm.Greys, )
ax2.set_zlim3d(-3,3)
plt.title("Outliers: %s%%" % Q)
plt.savefig(r"fig/isnn2019_surfplot_%s_%s.eps" % (n_input, n_hidden))
plt.show()

In [ ]:
n_param = nn_orig.n_param
nn_bp = create_nn(n_input, n_hidden)
nn_bp.param[:] = np.random.random(n_param)
nn_bp_o = create_nn(n_input, n_hidden)
nn_bp_o.param[:] = np.random.random(n_param)
nn_sqrt_o = create_nn(n_input, n_hidden)
nn_sqrt_o.param[:] = np.random.random(n_param)
nn_logsq_o = create_nn(n_input, n_hidden)
nn_logsq_o.param[:] = np.random.random(n_param)
nn_wm_o = create_nn(n_input, n_hidden)
nn_wm_o.param[:] = np.random.random(n_param)
nn_m_o = create_nn(n_input, n_hidden)
nn_m_o.param[:] = np.random.random(n_param)

In [ ]:
# er_bp = erisk(X, Y_s, nn_bp, loss_func_sq)
# fg_bp = erm_fg(er_bp, h=0.001, tol=1.0e-8, verbose=0, n_restart=5)

# plt.plot(fg_bp.lvals)
# plt.show()

In [ ]:
er_bp_o = erisk(X, Y_o, nn_bp_o, loss_func_sq)
fg_bp_o = erm_fg(er_bp_o, h=0.01, tol=1.0e-8, verbose=0, n_restart=5)

plt.plot(fg_bp_o.lvals)
plt.show()

In [ ]:
er_sqrt_o = erisk(X, Y_o, nn_sqrt_o, loss_func_sqrt)
fg_sqrt_o = erm_fg(er_sqrt_o, h=0.01, tol=1.0e-8, verbose=0, n_restart=5)

plt.plot(fg_sqrt_o.lvals)
plt.show()

In [ ]:
er_logsq_o = erisk(X, Y_o, nn_logsq_o, loss_func_logsq)
fg_logsq_o = erm_fg(er_logsq_o, h=0.01, tol=1.0e-8, verbose=0, n_restart=5)

plt.plot(fg_logsq_o.lvals)
plt.show()

In [ ]:
er_wm_o = erisk(X, Y_o, nn_wm_o, loss_func_sq)
fg_wm_o = fg(er_wm_o, h=0.005, tol=1.0e-9)
alpha = 0.999-q
avg_wm_o = averager_it(func.QuantileFunc(alpha, func.Sqrt(0.001)))
wg_wm_o = weights.WMWeights(avg_wm_o, er_wm_o)
irgd_wm_o = erm_irgd(fg_wm_o, wg_wm_o, n_iter=22, verbose=0)

plt.figure(figsize=(5,4))
plt.plot(irgd_wm_o.lvals)
plt.show()

In [ ]:
avg22 = averager_it(func.Quantile_Sqrt(alpha, 0.001))
avg22 = avragg.WMAverage(avg22)
mr2 = mrisk(X, Y, nn_m_o, loss_func_sq, avg22)
alg_fg2 = erm_fg(mr2, h=0.005, tol=1.0e-9, n_iter=1000, n_restart=5,)

plt.figure(figsize=(5,4))
plt.plot(alg_fg2.lvals)
plt.show()

In [ ]:
result[Q] = {
    'nn':(
        nn_bp_o, nn_sqrt_o, nn_logsq_o, nn_wm_o, nn_m_o),
    'title':(
        'BP | %s%%' % Q,
        'BP $\sqrt{1+x^2}$ | %s%%' % Q,
        'BP $\ln(1+x^2)$ | %s%%' % Q,
        'WM-BP | %s%%' % Q,
        'M-BP | %s%%' % Q,
    ),
    'b':0.1
}

In [ ]:
K = 5
m_q = len(result)
plt.figure(figsize=(K*3.5, m_q * 3.5))
for j, Q in enumerate(result):
    for i in range(K):
        current = result[Q]
        b = current['b']
        plt.subplot(m_q, K, K*j+i+1)
        _ffnn = current['nn'][i]
        Y1 = np.array(_ffnn.evaluate_all(X))
        title = current['title'][i]
        plt.title(title)
        V = np.abs(Y-Y1)
        I = np.argsort(V)
        I = I[::-1]
        Y2 = [np.min(Y1), np.max(Y1)]
        plt.plot(Y2, Y2, c='k')
        plt.scatter(Y[I], Y1[I], c=V[I], s=4, cmap=plt.cm.gray)
        c = sum(V < b) / len(V) * 100
        y_min, y_max = plt.xlim()
        plt.text(y_min, Y1.max(), r'$\{|err|<%.1f\}\to%s$%%' % (b, int(c)))
        if i == 0:
            plt.ylabel("predicted target")
        if j == m_q-1:
            plt.xlabel("original target")
        if j < m_q-1:
            plt.xticks([], [])
        plt.minorticks_on()
plt.tight_layout()
plt.savefig(r"fig/isnn2019_regr_%s_%s.eps" % (n_input, n_hidden))
plt.show()

In [ ]:
K = 5
m_q = len(result)
plt.figure(figsize=(K*3.0, m_q * 2.75))
for j, Q in enumerate(result):
    for i in range(K):
        current = result[Q]
        b = current['b']
        plt.subplot(m_q, K, K*j+i+1)
        _ffnn = current['nn'][i]
        Y1 = np.array(_ffnn.evaluate_all(X))
        title = current['title'][i]
        plt.title(title)
        V = np.abs(Y-Y1)
        I = np.argsort(V)
        I = I[::-1]
        #Y2 = [np.min(Y1), np.max(Y1)]
        #plt.plot(Y2, Y2, c='k')
        #plt.scatter(Y[I], Y1[I], c=V[I], s=4, cmap=plt.cm.gray)
        plt.hist(V, bins="auto", color='grey', edgecolor='k', density=True)
        _, h_max = plt.ylim()
        plt.vlines(np.mean(V), 0, h_max, colors='k')
        c = sum(V < b) / len(V) * 100
        y_min, y_max = plt.xlim()
        #plt.text(y_min, Y1.max(), r'$\{|err|<%.1f\}\to%s$%%' % (b, int(c)))
        if i == 0:
            plt.ylabel("predicted target")
        if j == m_q-1:
            plt.xlabel("original target")
#         if j < m_q-1:
#             plt.xticks([], [])
        plt.minorticks_on()
plt.tight_layout()
plt.savefig(r"fig/isnn2019_regr_hist_%s_%s.eps" % (n_input, n_hidden))
plt.show()

In [ ]:
K = 5
m_q = len(result)
lstyles = ['--', '-.', ':', '-', '-']
plt.figure(figsize=(4.0, 3.5))
current = result[Q]
b = current['b']
for i in range(K):
    _ffnn = current['nn'][i]
    Y1 = np.array(_ffnn.evaluate_all(X))
    title = current['title'][i]
    #plt.title(title)
    V = np.abs(Y-Y1)
    Vs = np.sort(V)
    plt.plot(Vs, c='k', linestyle=lstyles[i], label = title)
plt.tight_layout()
plt.ylim(0, 3.0)
plt.legend()
plt.savefig(r"fig/isnn2019_regr_err_%s_%s." % (n_input, n_hidden))
plt.show()